# Import Required Libraries
Import Pandas for data manipulation.

In [14]:
import pandas as pd

# Load CSV File into DataFrame
Use Pandas to load the CSV file 'CSVs\date_validation.csv' into a DataFrame named df_val.

In [15]:
# Load CSV File into DataFrame
df_val = pd.read_csv('CSVs\\date_validation.csv')
df_val['inicio_carga'] = pd.to_datetime(df_val['inicio_carga'])
df_val['fecha_visacion'] = pd.to_datetime(df_val['fecha_visacion'])

datetime_columns = ['fecha_recepcion', 'fecha_publicacion']
for column in datetime_columns:
    df_val[column] = pd.to_datetime(df_val[column])
    df_val[column] = df_val[column] - pd.Timedelta(hours=4)
    df_val[column] = df_val[column].dt.strftime('%Y-%m-%d %H:%M:%S')
    df_val[column] = pd.to_datetime(df_val[column])
# Crear el diccionario de renombramiento
rename_dict = {col: f"{col}_utc0" for col in datetime_columns}
# Renombrar las columnas
df_val = df_val.rename(columns=rename_dict)



df_val = df_val.drop('fecha_emision_licencia', axis=1)
df_val['inicio_carga_mod'] = df_val['inicio_carga'].apply(lambda x: (x + pd.Timedelta(days=1)).replace(hour=0, minute=0, second=0) if x.hour >= 18 else x) 



# df_val.loc[:, ['delta_inicio_carga', 'delta_fecha_visacion', 'delta_fecha_recepcion', 'send_date', 'inicio_carga_mod']]
df_val


,folio_recepcion,fecha_visacion,fecha_recepcion_utc0,fecha_publicacion_utc0,inicio_carga,estado_final_lm,inicio_carga_mod
0,2024003050,2024-06-18,2024-06-17 12:13:00,2024-06-19 11:16:04,2024-06-19 14:48:19.667371,4,2024-06-19 14:48:19.667371
1,2024003236,2024-06-19,2024-06-17 20:01:00,2024-06-19 08:16:06,2024-06-19 11:45:26.883902,6,2024-06-19 11:45:26.883902
2,2024003307,2024-06-18,2024-06-18 14:42:00,2024-06-19 15:15:48,2024-06-19 18:01:05.301247,6,2024-06-20 00:00:00.301247
3,2024013137,2024-06-18,2024-06-17 10:58:00,2024-06-19 15:16:14,2024-06-19 13:31:24.336156,6,2024-06-19 13:31:24.336156
4,2024043057,2024-06-19,2024-06-18 13:28:00,2024-06-21 13:16:02,2024-06-21 16:01:40.673992,6,2024-06-21 16:01:40.673992
...,...,...,...,...,...,...,...
1136,2024240857,2024-06-19,2024-06-17 20:01:00,2024-06-19 10:22:36,2024-06-19 13:15:28.655740,6,2024-06-19 13:15:28.655740
1137,2024240858,2024-06-19,2024-06-17 20:01:00,2024-06-19 09:17:27,2024-06-19 12:45:28.910710,6,2024-06-19 12:45:28.910710
1138,2024240889,2024-06-19,2024-06-18 20:01:00,2024-06-19 16:16:33,2024-06-19 19:15:27.211221,6,2024-06-20 00:00:00.211221
1139,2024467283,2024-06-19,2024-06-17 02:10:00,2024-06-21 01:32:55,2024-06-19 18:02:15.710733,3,2024-06-20 00:00:00.710733


# Load Excel Sheet into DataFrame
Load the third sheet of the Excel file 'excels\19-06.xlsx' into a DataFrame named df_con using Pandas.

In [16]:
# Load the third sheet of the Excel file into a DataFrame
df_con = pd.read_excel('excels\\19-06.xlsx', sheet_name='estado_1')
df_con

,Folio Recepcion,Fecha
0,2024003236,2024-06-19 07:05:30
1,2024043057,2024-06-19 17:27:55
2,2024233012,2024-06-19 12:30:05
3,2024233018,2024-06-19 18:28:46
4,2024233024,2024-06-19 12:14:02
...,...,...
1545,2024240858,2024-06-19 08:08:21
1546,2024240858,2024-06-19 08:08:48
1547,2024240858,2024-06-19 08:09:31
1548,2024240889,2024-06-19 14:36:21


# Filter DataFrame by Latest Date
Filter df_con to keep only the records with the latest date available.

In [17]:
df_con.rename(columns={'Folio Recepcion': 'Folio' }, inplace=True)


df_con.rename(columns={'Fecha': 'send_date' }, inplace=True)


# Convertir la columna 'send_date' a formato send_datetime si aún no se ha hecho
df_con['send_date'] = pd.to_datetime(df_con['send_date'])

# Obtener la última fecha para cada folio
df_con['min_send_date'] = df_con.groupby('Folio')['send_date'].transform('min')


df_con['min_send_date'] = pd.to_datetime(df_con['min_send_date'])

# Filtrar el DataFrame
df_con_2 = df_con[df_con['send_date'] == df_con['min_send_date']]

df_con_2 = df_con.drop_duplicates(subset=['Folio'], keep='first').reset_index(drop=True)

 
# Convertir la columna 'send_date' a formato send_datetime si aún no se ha hecho
df_con['send_date'] = pd.to_datetime(df_con['send_date'])

# Obtener la última fecha para cada folio
df_con['min_send_date'] = df_con.groupby('Folio')['send_date'].transform('max')


df_con['min_send_date'] = pd.to_datetime(df_con['min_send_date'])

# Filtrar el DataFrame
df_con_2 = df_con[df_con['send_date'] == df_con['min_send_date']]

df_con_2 = df_con.drop_duplicates(subset=['Folio'], keep='first').reset_index(drop=True).drop(columns=['min_send_date'])

 

df_con_2
# df_con_filtered

,Folio,send_date
0,2024003236,2024-06-19 07:05:30
1,2024043057,2024-06-19 17:27:55
2,2024233012,2024-06-19 12:30:05
3,2024233018,2024-06-19 18:28:46
4,2024233024,2024-06-19 12:14:02
...,...,...
789,2024240854,2024-06-19 08:18:41
790,2024240857,2024-06-19 08:36:12
791,2024240858,2024-06-19 08:08:21
792,2024240889,2024-06-19 14:36:21


# MERGED DFS


In [44]:
merged_df = df_val.merge(df_con_2, left_on='folio_recepcion', right_on='Folio')
merged_df['send_date_mod'] = merged_df['send_date'].apply(lambda x: (x + pd.Timedelta(days=1)).replace(hour=0, minute=0, second=0) if x.hour >= 18 else x) 


matched_dates = merged_df[merged_df['send_date_mod'].dt.date == merged_df['inicio_carga_mod'].dt.date]
not_matched_dates = merged_df[merged_df['send_date_mod'].dt.date != merged_df['inicio_carga_mod'].dt.date]
# merged_df
day_19 = merged_df.loc[merged_df['fecha_recepcion_utc0'] < merged_df['send_date'], ['folio_recepcion','inicio_carga', 'send_date', 'fecha_recepcion_utc0', 'fecha_visacion']]

# Convertir columnas a datetime
merged_df['fecha_visacion'] = pd.to_datetime(merged_df['fecha_visacion'])
merged_df['send_date'] = pd.to_datetime(merged_df['send_date'])
merged_df['fecha_recepcion_utc0'] = pd.to_datetime(merged_df['fecha_recepcion_utc0'])
merged_df['inicio_carga_mod'] = pd.to_datetime(merged_df['inicio_carga_mod'])
merged_df['send_date_mod'] = pd.to_datetime(merged_df['send_date_mod'])

merged_df['delta_inicio_carga'] = (merged_df['inicio_carga_mod'] - merged_df['send_date'])
merged_df['delta_fecha_visacion'] = (merged_df['fecha_visacion'].dt.date - merged_df['send_date_mod'].dt.date)
merged_df['delta_fecha_visacion'] = merged_df['delta_fecha_visacion'].apply(lambda x: x.days)
merged_df['delta_fecha_recepcion'] = (merged_df['fecha_recepcion_utc0'] - merged_df['send_date'])

# not_matched_dates
# print(merged_df[merged_df['send_date_mod'].dt.day == 19]) 
# print('df with matched dates')
# merged_df.loc[:, ['folio_recepcion', 'inicio_carga', 'inicio_carga_mod', 'send_date', 'delta_inicio_carga']]
# print('mean of delta inicio carga vs send date')
# print(merged_df['delta_inicar_senddate'].mean())
# merged_df['delta_inicar_senddate'].max()
columns_to_show =['folio_recepcion', 'fecha_visacion', 'fecha_recepcion_utc0',
'fecha_publicacion_utc0',
'inicio_carga_mod', 'send_date', 'send_date_mod',
'delta_inicio_carga', 'delta_fecha_visacion', 'delta_fecha_recepcion']
merged_df.loc[ merged_df['send_date_mod'].dt.day > 19 ,columns_to_show ]

,folio_recepcion,fecha_visacion,fecha_recepcion_utc0,fecha_publicacion_utc0,inicio_carga_mod,send_date,send_date_mod,delta_inicio_carga,delta_fecha_visacion,delta_fecha_recepcion
3,2024233018,2024-06-19,2024-06-17 14:29:00,2024-06-21 14:16:26,2024-06-22 00:00:00.666783,2024-06-19 18:28:46,2024-06-20,2 days 05:31:14.666783,-1,-3 days +20:00:14
5,2024233027,2024-06-19,2024-06-17 17:12:00,2024-06-23 22:16:08,2024-06-22 00:00:00.239601,2024-06-19 21:11:42,2024-06-20,2 days 02:48:18.239601,-1,-3 days +20:00:18
6,2024233031,2024-06-19,2024-06-17 15:07:00,2024-06-23 22:16:17,2024-06-22 00:00:00.596290,2024-06-19 19:07:03,2024-06-20,2 days 04:52:57.596290,-1,-3 days +19:59:57
7,2024233053,2024-06-19,2024-06-17 14:25:00,2024-06-21 14:16:33,2024-06-21 17:31:02.457816,2024-06-19 18:25:08,2024-06-20,1 days 23:05:54.457816,-1,-3 days +19:59:52
18,2024233131,2024-06-19,2024-06-17 15:26:00,2024-06-23 22:17:00,2024-06-22 00:00:00.169960,2024-06-19 19:25:25,2024-06-20,2 days 04:34:35.169960,-1,-3 days +20:00:35
...,...,...,...,...,...,...,...,...,...,...
557,2024239138,2024-06-19,2024-06-13 18:00:00,2024-06-21 14:38:00,2024-06-22 00:00:00.615541,2024-06-19 22:00:23,2024-06-20,2 days 01:59:37.615541,-1,-7 days +19:59:37
565,2024239337,2024-06-19,2024-06-13 18:33:00,2024-06-21 15:20:14,2024-06-22 00:00:00.756342,2024-06-19 22:32:56,2024-06-20,2 days 01:27:04.756342,-1,-7 days +20:00:04
570,2024239476,2024-06-19,2024-06-13 19:21:00,2024-06-21 15:20:25,2024-06-22 00:00:00.024868,2024-06-19 23:21:15,2024-06-20,2 days 00:38:45.024868,-1,-7 days +19:59:45
571,2024239537,2024-06-19,2024-06-13 18:36:00,2024-06-21 15:20:33,2024-06-22 00:00:00.970530,2024-06-19 22:35:28,2024-06-20,2 days 01:24:32.970530,-1,-7 days +20:00:32


DIFERENCIA ENTRE FOLIOS PROPIOS Y FOLIOS CONSALUD